In [ ]:
import requests
year = 2022
day = 16

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open('cookie.txt', 'r') as file:
    session = file.read()

headers = {'cookie': f"session={session}"}

with open(f'day_{day}_input.txt', 'wb') as file:
    file.write(requests.get(url, headers=headers).content)
    
import os
if os.path.exists(f"day_{day}_test.txt"):
    test = [x.strip() for x in open(f'day_{day}_test.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
if os.path.exists(f"day_{day}_test_2.txt"):
    test_2 = [x.strip() for x in open(f'day_{day}_test_2.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
lines = [x.strip() for x in open(f'day_{day}_input.txt', 'r')]
for line in lines[:10]:
    print(line.strip())

In [457]:
import re

run_test = False

if run_test:
    case = test
else:
    case = lines
    
V = {}
L = []
for line in case:
    pattern = r"Valve (\S+) has flow rate=(\d+); tunnels? leads? to valves? ([\S, ]+)"
    groups = re.findall(pattern,line)
    s = groups[0][0]
    p = int(groups[0][1])
    ts = [x.strip() for x in groups[0][2].split(',')]
    V[s] = p
    for t in ts:
        L.append((s,t))

In [345]:
from pprint import pprint
#print(V)

# At each step, options are:
    # move, open
    
DP = dict()

num_lookups = 0
num_calcs = 0

def advance_time(state, current_pressure_released):
    global num_calcs, num_lookups
    position = state[0]
    time = state[1]
    current_open_valves = state[2]
    
    if time == 30:
        #assert False
        #print(current_pressure_released)
        return current_pressure_released
    
    if state in DP:
        num_lookups+=1
        return DP[state]
    else:
        num_calcs+=1
        current_pressure_released += sum([V[v] for v in current_open_valves])
        #print(state, current_pressure_released)

        # Move
        for l in [l[1] for l in L if l[0] == position]:
            n_state = (l, time+1, current_open_valves)
            advance_time(n_state, current_pressure_released)
            #DP[n_state] = current_pressure_released


        # Open
        for l in [l[0] for l in L if l[0] == position and l[0] not in current_open_valves and V[l[0]] > 0]:
            n_state = (position, time+1, current_open_valves + tuple([l]))
            advance_time(n_state, current_pressure_released)
        
        DP[state] = current_pressure_released
        #return DP[state]

advance_time(('AA',0,()),0)
# Too slow
#pprint(DP)
print(num_lookups,num_calcs)
print(max(DP.values()))

76198 73640
1651


In [443]:
print([v for v in V if V[v] > 0])

['AT', 'GQ', 'OC', 'TR', 'VK', 'YL', 'ES', 'XN', 'EY', 'YA', 'AZ', 'WZ', 'UI', 'JT', 'RK']


In [ ]:
from pprint import pprint
from collections import deque
    
DP = dict()

MAX_DURATION = 26

def advance_time(state, current_pressure_released):
    positions = state[0]
    time = state[1]
    current_open_valves = state[2]
        
    current_pressure_released = current_pressure_released + sum([V[v] for v in current_open_valves])
    if state in DP:
        if current_pressure_released <= DP[state]:
            return DP[state]
    
    DP[state] = current_pressure_released
    
    #all valves open - sprint to end.
    if len(current_open_valves) == len([v for v in V if V[v] > 0]):
        for i, t in enumerate(range(time+1,MAX_DURATION+1)):
            n_state = (positions, t, current_open_valves)
            if n_state not in DP:
                DP[n_state] = current_pressure_released + i * sum([V[v] for v in current_open_valves])
        return DP[state]

    if time == DURATION:
        return DP[state]
    
    searches = list()

    # both open
    if V[positions[0]] > 0 and V[positions[1]] > 0:
        n_state = (positions, time+1, tuple(sorted(set(current_open_valves + positions))))
        searches.append(n_state)

    #you move, elephant opens
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[0] for l in L if l[0] == positions[1] and V[positions[1]] > 0]:
            n_state = ((i,positions[1]), time+1, tuple(sorted(set(current_open_valves + tuple([j])))))
            searches.append(n_state)

    # elephant moves, you open
    for i in [l[0] for l in L if l[0] == positions[0] and V[positions[0]] > 0]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((positions[0],j), time+1, tuple(sorted(set(current_open_valves + tuple([i])))))
            searches.append(n_state)


    # both move
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((i,j), time+1, tuple(sorted(set(current_open_valves))))
            searches.append(n_state)

    # both stay
    n_state = (positions, time+1, tuple(sorted(set(current_open_valves))))
    advance_time(n_state, current_pressure_released)
    searches.append(n_state)

    for search in set(searches):#sorted(searches, key= lambda x: sum([V[v] for v in current_open_valves if v in x])):            
        advance_time(search, current_pressure_released)
        
    return DP[state]
        

DURATION = 26
advance_time((('AA','AA'), 0, ()),0)
#advance_time((('AA', 'EE'), 11, ('BB', 'CC', 'DD', 'EE', 'HH', 'JJ')),573)

#print(max(DP.values()))

for i in range(1,26+1):
    max_day = 0
    max_key = None
    for k,v in DP.items():
        if k[1] == i and v >= max_day:
            max_day = v
            max_key = k
    print(f"{i}: {max_day}, {max_key}")
# 2298 too high
# 2113 too low

In [386]:
from pprint import pprint
from collections import deque

    
DP = {(('AA','AA'),0,()):0}

def advance_time(state, current_pressure_released):
    positions = state[0]
    time = state[1]
    current_open_valves = state[2]

    current_pressure_released = current_pressure_released + sum([V[v] for v in current_open_valves])

    if state not in DP:
        DP[state] = current_pressure_released
        
    if current_pressure_released > DP[state]:
        DP[state] = current_pressure_released

    if time == DURATION:
        return []# DP[state]

    searches = list()

    if V[positions[0]] > 0 and V[positions[1]] > 0:
        n_state = (positions, time+1, tuple(sorted(set(current_open_valves + positions))))
        searches.append(n_state)

    #you move, elephant opens
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[0] for l in L if l[0] == positions[1] and V[l[0]] > 0]:
            n_state = ((i,positions[1]), time+1, tuple(sorted(set(current_open_valves + tuple([j])))))
            searches.append(n_state)

    # elephant moves, you open
    for i in [l[0] for l in L if l[0] == positions[0] and V[l[0]] > 0]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((positions[0],j), time+1, tuple(sorted(set(current_open_valves + tuple([i])))))
            searches.append(n_state)


    # both move
    for i in [l[1] for l in L if l[0] == positions[0]]:
        for j in [l[1] for l in L if l[0] == positions[1]]:
            n_state = ((i,j), time+1, tuple(sorted(set(current_open_valves))))
            searches.append(n_state)

        
        # both stay
        #n_state = (positions, time+1, tuple(sorted(set(current_open_valves))))
        #advance_time(n_state, current_pressure_released)
        #searches.append(n_state)
#         print(len(searches))
#         pprint(searches)
        

    #return [search for search in sorted(searches, key= lambda x: sum([V[v] for v in current_open_valves if v in x])), current_pressure_released]
    return set(searches), current_pressure_released

DURATION = 12

best = {(('AA','AA'),0,()):0}
for i in range(DURATION):
    to_search = []
    for state in best:
        searches = advance_time(state,0)
        to_search.extend(searches[0])
        #pprint(searches)
    best = to_search.copy()
    

for i in range(1,DURATION):
    max_day = 0
    max_key = None
    for k,v in DP.items():
        if k[1] == i and v >= max_day:
            max_day = v
            max_key = k
    print(f"{i}: {max_day}, {max_key}")

KeyboardInterrupt: 